In [2]:
# this file contains code for compute features for assign cell cycle state of each celltype
# features include: near%	mitotic%	farAvg	repli score, the result is saved in the file:
# HiRES_emb_metadata.xlsx

import sys
sys.path.append("/shareb/zliu/analysis/hires_gastrulation/")
from CHARMtools import cellcycle
from CHARMtools import CHARMio

import glob
import pandas as pd

from concurrent import futures
from functools import partial

In [3]:
PAIRS_PATH = "../../../submitfiles/pairs_clean3/" # where you kept downloaded pairs.gz files from GEO

#read excel
metadata = pd.read_excel("/shareb/zliu/analysis/hires_mouse_dev/submission/submitfiles/HiRES_emb_metadata.xlsx")
# keep only cellname column since we don't need other information
metadata = metadata[["Cellname"]]
metadata["pairs"] = metadata["Cellname"].apply(lambda x: "../../../submitfiles/pairs_clean3/"+x+".pairs.gz")

In [4]:
metadata["pairs"] = metadata["Cellname"].apply(lambda x: "../../../submitfiles/pairs_clean3/"+x+".pairs.gz")

In [ ]:
ordering = cellcycle.calc_cellcycle_ordering(metadata,threads=100)
describe = cellcycle.calc_contact_describe(metadata,threads=100)
repliscore = cellcycle.calc_repli_score(metadata,repli_chipf="/shareb/zliu/analysis/hires_gastrulation/CHARMtools/ref/mm10_repli_chip.wig.gz",threads=100)

In [5]:
res = pd.concat([ordering,describe,repliscore],axis=1)
#res.to_csv("DNA_cellcycle.tsv",sep="\t",index=None)

In [13]:
res

,Cellname,pairs,order_index,near_p,mitotic_p,farAvg,NaganoCellcycle,repli_score,annote_ratio
0,GasaE751001,../../../submitfiles/pairs_clean3/GasaE751001....,6337,0.512112,0.202479,2.000648e+07,G1,0.585603,0.994522
1,GasaE751002,../../../submitfiles/pairs_clean3/GasaE751002....,1728,0.685396,0.119005,2.724219e+07,early/mid-S,0.618245,0.995242
2,GasaE751003,../../../submitfiles/pairs_clean3/GasaE751003....,1669,0.738354,0.121796,2.288031e+07,early/mid-S,0.548804,0.994558
3,GasaE751004,../../../submitfiles/pairs_clean3/GasaE751004....,7037,0.602727,0.341371,1.385462e+07,Pre-M,0.529733,0.993867
4,GasaE751005,../../../submitfiles/pairs_clean3/GasaE751005....,7081,0.405741,0.228987,1.879708e+07,G1,0.492611,0.992573
...,...,...,...,...,...,...,...,...,...
7464,OrgeEX053380,../../../submitfiles/pairs_clean3/OrgeEX053380...,1093,0.726202,0.122614,3.359666e+07,early/mid-S,0.577465,0.995228
7465,OrgeEX053381,../../../submitfiles/pairs_clean3/OrgeEX053381...,4878,0.576129,0.140135,3.913699e+07,G1,0.521149,0.993799
7466,OrgeEX053382,../../../submitfiles/pairs_clean3/OrgeEX053382...,628,0.733671,0.110580,3.218927e+07,early/mid-S,0.587758,0.994891
7467,OrgeEX053383,../../../submitfiles/pairs_clean3/OrgeEX053383...,6608,0.471231,0.182294,2.471220e+07,G1,0.524906,0.993949


In [ ]:
with futures.ProcessPoolExecutor(100) as pool:
    res = pool.map(cellcycle.dis_counts,metadata["pairs"])
ares = list(res)
cdps = pd.DataFrame(ares)
cdps.columns = cdps.columns.astype("string")
cdps.index = metadata.reset_index().index